In [ ]:
! pip install langchain
! pip install langchain_community
! pip install dashscope --upgrade
! pip install --upgrade --quiet  opencv-python scikit-image
! pip install langchain_openai
! pip install gradio_tools

! pip install json5
! pip install transformers_stream_generator einops
! pip install accelerate

In [ ]:

import json
import os

import json5
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig


from langchain_community.chat_models.tongyi import ChatTongyi

os.environ["DASHSCOPE_API_KEY"] = "your-qwen-key"
model = ChatTongyi(temperature=0.0,model_name = 'qwen1.5-110b-chat')

# history = []
# messages=[{'role':'system', 'content':'You are a helpful assistant.'}]


# 将一个插件的关键信息拼接成一段文本的模版。
TOOL_DESC = """{name_for_model}: Call this tool to interact with the {name_for_human} API. What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters}"""

# ReAct prompting 的 instruction 模版，将包含插件的详细信息。
PROMPT_REACT = """Answer the following questions as best you can. You have access to the following APIs:

{tools_text}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tools_name_text}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {query}"""




In [ ]:
#
# 本示例代码的入口函数。
#
# 输入：
#   prompt: 用户的最新一个问题。
#   history: 用户与模型的对话历史，是一个 list，
#       list 中的每个元素为 {"user": "用户输入", "bot": "模型输出"} 的一轮对话。
#       最新的一轮对话放 list 末尾。不包含最新一个问题。
#   list_of_plugin_info: 候选插件列表，是一个 list，list 中的每个元素为一个插件的关键信息。
#       比如 list_of_plugin_info = [plugin_info_0, plugin_info_1, plugin_info_2]，
#       其中 plugin_info_0, plugin_info_1, plugin_info_2 这几个样例见本文档前文。
#
# 输出：
#   模型对用户最新一个问题的回答。
#
def llm_with_plugin(prompt: str, history, list_of_plugin_info=()):
    chat_history = [(x['user'], x['bot']) for x in history] + [(prompt, '')]

    planning_prompt = build_input_text(chat_history, list_of_plugin_info)

    text = ''
    while True:
        print("\033[32m" + "---等待LLM返回...\n" + planning_prompt + "\033[0m", flush=True)

        output = text_completion(planning_prompt + text, stop_words=['Observation:', 'Observation:\n'])

        print("\033[34m" + "output:" + output + "\033[0m")
        action, action_input, output = parse_latest_plugin_call(output)
        if action:  # 需要调用插件
            # action、action_input 分别为需要调用的插件代号、输入参数
            # observation是插件返回的结果，为字符串
            observation = call_plugin(action, action_input)
            output += f'\nObservation: {observation}\nThought:'
            text += output
        else:  # 生成结束，并且不再需要调用插件
            text += output
            break

    new_history = []
    new_history.extend(history)
    new_history.append({'user': prompt, 'bot': text})
    return text, new_history


# 将对话历史、插件信息聚合成一段初始文本
def build_input_text(chat_history, list_of_plugin_info) -> str:
    # 候选插件的详细信息
    tools_text = []
    for plugin_info in list_of_plugin_info:
        tool = TOOL_DESC.format(
            name_for_model=plugin_info["name_for_model"],
            name_for_human=plugin_info["name_for_human"],
            description_for_model=plugin_info["description_for_model"],
            parameters=json.dumps(plugin_info["parameters"], ensure_ascii=False),
        )
        if plugin_info.get('args_format', 'json') == 'json':
            tool += " Format the arguments as a JSON object."
        elif plugin_info['args_format'] == 'code':
            tool += ' Enclose the code within triple backticks (`) at the beginning and end of the code.'
        else:
            raise NotImplementedError
        tools_text.append(tool)
    tools_text = '\n\n'.join(tools_text)

    # 候选插件的代号
    tools_name_text = ', '.join([plugin_info["name_for_model"] for plugin_info in list_of_plugin_info])

    im_start = '<|im_start|>'
    im_end = '<|im_end|>'
    prompt = f'{im_start}system\nYou are a helpful assistant.{im_end}'
    for i, (query, response) in enumerate(chat_history):
        if list_of_plugin_info:  # 如果有候选插件
            # 倒数第一轮或倒数第二轮对话填入详细的插件信息，但具体什么位置填可以自行判断
            if (len(chat_history) == 1) or (i == len(chat_history) - 2):
                query = PROMPT_REACT.format(
                    tools_text=tools_text,
                    tools_name_text=tools_name_text,
                    query=query,
                )
        query = query.lstrip('\n').rstrip()  # 重要！若不 strip 会与训练时数据的构造方式产生差异。
        response = response.lstrip('\n').rstrip()  # 重要！若不 strip 会与训练时数据的构造方式产生差异。
        # 使用续写模式（text completion）时，需要用如下格式区分用户和AI：
        prompt += f"\n{im_start}user\n{query}{im_end}"
        prompt += f"\n{im_start}assistant\n{response}{im_end}"

    assert prompt.endswith(f"\n{im_start}assistant\n{im_end}")
    prompt = prompt[: -len(f'{im_end}')]
    return prompt


def text_completion(input_text: str, stop_words) -> str:  #
    im_end = '<|im_end|>'
    if im_end not in stop_words:
        stop_words = stop_words + [im_end]
    # stop_words_ids = [tokenizer.encode(w) for w in stop_words] # transformers stop word 按照token方式传入

    # TODO: 增加流式输出的样例实现
    # input_ids = torch.tensor([tokenizer.encode(input_text)]).to(model.device)
    output = model.invoke(input_text, stop=stop_words)
    # output = output.tolist()[0]
    output = output.content
    # output = tokenizer.decode(output, errors="ignore")
    # assert output.startswith(input_text) # 判断是否需要续写
    # output = output[len(input_text) :].replace('<|endoftext|>', '').replace(im_end, '')

    # TODO: 增强 fuction calling 能力
    # for stop_str in stop_words:
    #     idx = output.find(stop_str)
    #     if idx != -1:
    #         output = output[: idx + len(stop_str)]
    return output  # 续写 input_text 的结果，不包含 input_text 的内容


def parse_latest_plugin_call(text):
    plugin_name, plugin_args = '', ''
    i = text.rfind('\nAction:')
    j = text.rfind('\nAction Input:')
    k = text.rfind('\nObservation:')
    if 0 <= i < j:  # If the text has `Action` and `Action input`,
        if k < j:  # but does not contain `Observation`,
            # then it is likely that `Observation` is ommited by the LLM,
            # because the output text may have discarded the stop word.
            text = text.rstrip() + '\nObservation:'  # Add it back.
        k = text.rfind('\nObservation:')
        plugin_name = text[i + len('\nAction:') : j].strip()
        plugin_args = text[j + len('\nAction Input:') : k].strip()
        text = text[:k]
    return plugin_name, plugin_args, text


#
# 输入：
#   plugin_name: 需要调用的插件代号，对应 name_for_model。
#   plugin_args：插件的输入参数，是一个 dict，dict 的 key、value 分别为参数名、参数值。
# 输出：
#   插件的返回结果，需要是字符串。
#   即使原本是 JSON 输出，也请 json.dumps(..., ensure_ascii=False) 成字符串。
#
def call_plugin(plugin_name: str, plugin_args: str) -> str:
    #
    # TODO: add RAG as a toll
    #
    if plugin_name == 'google_search':
        os.environ["SERPER_API_KEY"] = "your-google-key"
        # os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY", default='')
        from langchain_community.utilities import GoogleSerperAPIWrapper
        search = GoogleSerperAPIWrapper()

        return search.run(json5.loads(plugin_args)['search_query'])

    elif plugin_name == 'human_assistant':
        # from langchain.agents import load_tools
        # human_tool = load_tools(tool_names=["human"])[0]
        # "看起来您没有提到需要多大尺寸的图片，如果有需求的话请告诉我: 1. 780*1024 长版 2. 1024 * 780 横版 3.暂时无所谓"
        user_input = input(json5.loads(plugin_args)['ask_query'])

        # res = human_tool.run(json5.loads(plugin_args)['ask_query'])

        return user_input

    elif plugin_name == 'nice_prompt':
        from gradio_tools import (StableDiffusionPromptGeneratorTool,
                      TextToVideoTool)

        nice_prompt = StableDiffusionPromptGeneratorTool().langchain
        prompt = json5.loads(plugin_args)["prompt"]
        return nice_prompt.run(prompt)

    elif plugin_name == 'image_gen':
        import urllib.parse

        prompt = json5.loads(plugin_args)["prompt"]
        prompt = urllib.parse.quote(prompt)
        return json.dumps({'image_url': f'https://image.pollinations.ai/prompt/{prompt}'}, ensure_ascii=False)

    elif plugin_name == 'video_gen':
        from gradio_tools import (StableDiffusionPromptGeneratorTool,
              TextToVideoTool)
        import urllib.parse

        prompt = json5.loads(plugin_args)["prompt"]
        video_gen = TextToVideoTool().langchain
        # 'a cute dog swimming in the sea'
        video_path = video_gen.run(prompt)

        return video_path



    else:
        raise NotImplementedError




def test():
    tools = [
        {
            'name_for_human': '谷歌搜索',
            'name_for_model': 'google_search',
            'description_for_model': '谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。',
            'parameters': [
                {
                    'name': 'search_query',
                    'description': '搜索关键词或短语',
                    'required': True,
                    'schema': {'type': 'string'},
                }
            ],
        },
        {
            'name_for_human': '人类帮助工具',
            'name_for_model': 'human_assistant',
            'description_for_model': '人类帮助工具是一个和人类交互的过程, 如果有任何你觉得需要和用户沟通的问题，请使用它。',
            'parameters': [
                {
                    'name': 'ask_query',
                    'description': '需要和人类辅助确认的问题',
                    'required': True,
                    'schema': {'type': 'string'},
                }
            ],
        },
        {
            'name_for_human': 'prompt优化',
            'name_for_model': 'nice_prompt',
            'description_for_model': 'prompt优化是一个对文生图输入prompt的优化器，输入原始prompt，输出优化后的prompt。',
            'parameters': [
                {
                    'name': 'prompt',
                    'description': '原始的prompt',
                    'required': True,
                    'schema': {'type': 'string'},
                }
            ],
        },
        {
            'name_for_human': '文生图',
            'name_for_model': 'image_gen',
            'description_for_model': 'Before you use this api,Confirm with a human whether optimization of the input prompt is needed。文生图是一个AI绘画（图像生成）服务，输入文本描述，返回根据文本作画得到的图片的URL。',
            'parameters': [
                {
                    'name': 'prompt',
                    'description': '英文关键词，描述了希望图像具有什么内容',
                    'required': True,
                    'schema': {'type': 'string'},
                }
            ],
        },
        {
            'name_for_human': '文生视频',
            'name_for_model': 'video_gen',
            'description_for_model': '文生视频是一个AI视频（视频生成）服务，输入文本描述，返回根据文本生成视频得到的本地mp4文件路径。',
            'parameters': [
                {
                    'name': 'prompt',
                    'description': '英文关键词，描述了希望视频生成的内容',
                    'required': True,
                    'schema': {'type': 'string'},
                }
            ],
        },
    ]
    history = []
    # example：
    # 通识类问题：['你好', '还呗是一家什么公司', '再搜下这家公司的评价', '告诉我这家公司的官网']
    # 文生图：['你好', '给我画个可爱的小猫吧，最好是黑猫']
    # 文生视频：['你好', '一个可爱的金毛狗在海边游泳']
    for query in ['你好，帮我检索还呗的信息，以json返回']:
        print(f"User's Query:\n{query}\n")
        response, history = llm_with_plugin(prompt=query, history=history, list_of_plugin_info=tools)
        print(f"Qwen's Response:\n{response}\n")


In [ ]:
test()

User's Query:
你好，帮我检索还呗的信息，以json返回

---等待LLM返回...
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Answer the following questions as best you can. You have access to the following APIs:

google_search: Call this tool to interact with the 谷歌搜索 API. What is the 谷歌搜索 API useful for? 谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。 Parameters: [{"name": "search_query", "description": "搜索关键词或短语", "required": true, "schema": {"type": "string"}}] Format the arguments as a JSON object.

human_assistant: Call this tool to interact with the 人类帮助工具 API. What is the 人类帮助工具 API useful for? 人类帮助工具是一个和人类交互的过程, 如果有任何你觉得需要和用户沟通的问题，请使用它。 Parameters: [{"name": "ask_query", "description": "需要和人类辅助确认的问题", "required": true, "schema": {"type": "string"}}] Format the arguments as a JSON object.

nice_prompt: Call this tool to interact with the prompt优化 API. What is the prompt优化 API useful for? prompt优化是一个对文生图输入prompt的优化器，输入原始prompt，输出优化后的prompt。 Parameters: [{"name": "prompt", "description": "

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()

search.run('还呗是一家什么公司')

'还呗是一家专业的互联网金融公司，致力于为个人和企业提供全方位的金融服务。 作为一家成熟的金融科技公司，还呗拥有强大的技术和风控团队，能够为用户提供快捷、便利的借贷服务。 随着借贷市场的不断扩大，债务催收问也日益突出。'

In [ ]:
video_path = '/tmp/gradio/e82ebd13337f100a36f175a9c3379ba8a88513e1/tmpbbhqx4td.mp4'

In [ ]:
# prompt: 播放video_path的视频

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, width=640):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={width} controls><source src="{video_url}"></video>""")

show_video(video_path)
